In [1]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pickle

# def load_sequences(part):
#     with open('drive/My Drive/Machine_Learning-prj/vietnamese_language_model/data/sequences'+ str(part) + '.0', 'rb') as f:
#         sequences = pickle.load(f)
#     return sequences[:1000000]

In [0]:
# sequences = load_sequences(1)

In [0]:
# len(sequences)

In [0]:
# import keras

# tokenizer = keras.preprocessing.text.Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\]^`{|}~ ')

In [0]:
# tokenizer.fit_on_texts(sequences)

In [0]:
# with open('drive/My Drive/Machine_Learning-prj/vietnamese_language_model/tokenizer_S1.pkl', 'wb') as f:
#     pickle.dump(tokenizer, f)

In [0]:
# sequences_digit = tokenizer.texts_to_sequences(sequences)

In [0]:
# with open('drive/My Drive/Machine_Learning-prj/vietnamese_language_model/sequences_digit_S1.pkl', 'wb') as f:
#     pickle.dump(sequences_digit, f)

In [0]:
#del sequences

In [11]:
 with open('drive/My Drive/Machine_Learning-prj/vietnamese_language_model/tokenizer_S1.pkl', 'rb') as f:
    tokenizer = pickle.load(f)

Using TensorFlow backend.


In [0]:
with open('drive/My Drive/Machine_Learning-prj/vietnamese_language_model/sequences_digit_S1.pkl', 'rb') as f:
    sequences_digit = pickle.load(f)

In [0]:
vocab_size = len(tokenizer.word_index) + 1

In [0]:
import numpy as np

def data_generator(sequences_digit, batch_size):
    while True:
        batch_paths = np.random.choice(a = len(sequences_digit), size = batch_size)
        input = []
        output = []
        
        for i in batch_paths:
            input.append(sequences_digit[i][:-1])
            output.append(sequences_digit[i][-1])
        input = np.array(input)
        output = keras.utils.to_categorical(output, num_classes=vocab_size)
        output = np.array(output)
        
        yield (input, output)
        

In [0]:
from keras.models import Model, Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, GRU, Dense, Input, Add, Concatenate, Reshape, Lambda, BatchNormalization, Dropout, Embedding, LSTM
from keras.optimizers import SGD, RMSprop
import keras

In [24]:
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=50))
model.add(BatchNormalization())
model.add(LSTM(512, return_sequences=True))
model.add(LSTM(512))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(vocab_size, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 50, 50)            1733100   
_________________________________________________________________
batch_normalization_3 (Batch (None, 50, 50)            200       
_________________________________________________________________
lstm_3 (LSTM)                (None, 50, 512)           1153024   
_________________________________________________________________
lstm_4 (LSTM)                (None, 512)               2099200   
_________________________________________________________________
dense_3 (Dense)              (None, 100)               51300     
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 100)               400       
__________

In [0]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [22]:
from keras.models import load_model

print("Loading pre-trained weight")
model.load_weights('drive/My Drive/Machine_Learning-prj/weights-training-improvement-languagemodel.hdf5')

Loading pre-trained weight


In [0]:
from keras.callbacks import ModelCheckpoint

filepath = 'drive/My Drive/Machine_Learning-prj/weights-training-improvement-languagemodel.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [0]:
batch_size=512

In [21]:
model.fit_generator(generator=data_generator(sequences_digit, batch_size), steps_per_epoch=(len(sequences_digit)//batch_size) , epochs=100, verbose=1, callbacks=callbacks_list)

Epoch 1/100
1953/1953 [==============================] - 744s 381ms/step - loss: 3.2499 - acc: 0.3510

Epoch 00001: acc improved from -inf to 0.35099, saving model to drive/My Drive/Machine_Learning-prj/weights-training-improvement-languagemodel.hdf5
Epoch 2/100
1953/1953 [==============================] - 739s 379ms/step - loss: 3.2380 - acc: 0.3523

Epoch 00002: acc improved from 0.35099 to 0.35228, saving model to drive/My Drive/Machine_Learning-prj/weights-training-improvement-languagemodel.hdf5
Epoch 3/100
1953/1953 [==============================] - 739s 378ms/step - loss: 3.2206 - acc: 0.3551

Epoch 00003: acc improved from 0.35228 to 0.35511, saving model to drive/My Drive/Machine_Learning-prj/weights-training-improvement-languagemodel.hdf5
Epoch 4/100
1953/1953 [==============================] - 740s 379ms/step - loss: 3.2057 - acc: 0.3569

Epoch 00004: acc improved from 0.35511 to 0.35693, saving model to drive/My Drive/Machine_Learning-prj/weights-training-improvement-langua

KeyboardInterrupt: ignored

In [0]:
model.save('drive/My Drive/Machine_Learning-prj/vietnamese_language_model/39_weight_language_model.h5')